## 1.导入依赖库

In [5]:





import torch.utils.data
import torchvision.utils as vutils

In [7]:
import os
import torch
import torchvision.utils as vutils
import numpy as np
from models import model_utils

## 2.创建实验参数

In [8]:
from margs import myArg
args=myArg()

## 3.导入数据集

In [9]:
from datasets.DiLiGenT_main import DiLiGenT_main
test_set  = DiLiGenT_main(args, 'test')

[test Data] 	10 objs 96 lights. Root: data/datasets/DiLiGenT/pmsData
Files for intensity: light_intensities.txt


In [10]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=args.test_batch,
                                          num_workers=args.workers, pin_memory=args.cuda, shuffle=False)

In [11]:
print(args.retrain)

data/models/PS-FCN_B_S_32.pth.tar


## 4.导入模型

In [12]:
from models.PS_FCN_run import PS_FCN
#输入图片数量和光线数量
other = {'img_num': args.in_img_num, 'in_light': args.in_light}

#得到输入的通道数，如果输入光照则为6通道，否则为3通道
in_c = model_utils.getInputChanel(args)
model = PS_FCN(args.fuse_type, args.use_BN, in_c, other)
model = model.cuda()
#导入预训练模型
model_utils.loadCheckpoint(args.retrain, model, cuda=args.cuda)

[Network Input] Color image as input
[Network Input] Adding Light direction as input
[Network Input] Input channel: 6
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1


## 5.数据前向传播

In [13]:
model.eval()
split='test'
with torch.no_grad():
    #loader就是DataLoade处理的数据，i是第几个obj，sample是第i个obj对应的item
    for i, sample in enumerate(test_loader):
        #这一步img的形状没变，就是改成tensor了
        data = model_utils.parseData(args, sample, None, split)
        #这个函数应该是得到输入的吧，直接传给模型了
        input = model_utils.getInput(args, data)

        #得到模型输出数据
        out_var = model(input)
        #计算准确率，应该是一个loss之类的
        acc = eval_utils.calNormalAcc(data['tar'].data, out_var.data, data['m'].data) 
        
        pred = (out_var.data + 1) / 2
        masked_pred = pred * data['m'].data.expand_as(out_var.data)
        #路径为当前模型下
#         save_dir = os.path.join(os.path.join(os.path.dirname(args.retrain), 'run_model'), split)
        save_name ='deep_norm{}.png'.format(i)
        #保存图片
        vutils.save_image(masked_pred, os.path.join("results", save_name))

        

[26 84  2 55 68 81 16 78 54 66 53 88 71 13  7 30 22 24 33  8 43 62  3 86
 45 48  6 95 80 75 60 79]
******************读取图片中*********************


C:\Users\DELL\Desktop\PS-FCN-master\utils\eval_utils.py:12: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:28.)
  ang_valid   = angular_map[mask.narrow(1, 0, 1).squeeze(1).byte()]


[40 31 46 59 78 74 65 44 80 85 62 27 91 77 32 56 39 10  2 38 54 58 19 90
 95 35 33 48 84 67 55 36]
******************读取图片中*********************
[36 28 54 23 16 80  2 25 84 13 59 88 76 14  0 21  3 27 74 71 11 81 30 29
  5 73 56 95 83  1 18 24]
******************读取图片中*********************
[61 23 94  9 25  6 79 71 57 64 15 77 58 70 81 45 78 89 42 93 12  8  4 76
 34 17 68 82 40  5 13 31]
******************读取图片中*********************
[81 14 13 53 59 20 37 10 63  2 16 24 35 92 47 54 26 90 71 65 77 83 41 93
  5 67 19 39 29 27  4 79]
******************读取图片中*********************
[42 57 48 32 67 22 10 20 17 28 35 34 54 24 39 60 50 83 12 66 91 46 23 13
 59 68 37 84  6 92 26 40]
******************读取图片中*********************
[55 20 23 30  0 38 37 64 39 59 17 82 61 34 51 44 92 89 54 13 91 70 86 16
  9 45 52 27 47 29 32 53]
******************读取图片中*********************
[20 82 77 73 37 15 52 36 26 33 13 22 95 12 59 40 11 71 18 93  2 17 62  5
 45 91 58 30 51 49 89 27]
******************读取图片中**************